In [ ]:
# Access MinIO files
param_minio_endpoint = "scruffy.lab.uvalight.net:9000"
param_minio_user_prefix = "zhanqing2016@gmail.com"  # Your personal folder in the naa-vre-user-data bucket in MinIO
secret_minio_access_key = "sFmE1jsm5hjJBBGh5RBL"
secret_minio_secret_key = "pczCG6FRpXQEtad7lAvXv00iCYFd5Dpa1g8GOWzR"

In [ ]:
# Temp_SED_data
# ---
# NaaVRE:
#  cell:
#   inputs: []
#   outputs: [] 
#   secrets:
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies: 
#    - name: aws.s3
# ...


#   inputs:
#    - acolite_processing: List
# =========================
# Configure MinIO access
# =========================
param_minio_endpoint = "scruffy.lab.uvalight.net:9000"
param_minio_user_prefix = "zhanqing2016@gmail.com"  # Your personal folder in the naa-vre-user-data bucket in MinIO
secret_minio_access_key = ""
secret_minio_secret_key = ""


Sys.setenv("AWS_S3_ENDPOINT" = param_minio_endpoint,
           "AWS_DEFAULT_REGION" = "nl-uvalight",
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# =========================
# Load required libraries
# =========================
library(aws.s3)
library(devtools)
library(TempSED)
library(ReacTran)
library(pracma)
library(parallel)

# List existing buckets: get a list of all available buckets
bucketlist()

# List files in bucket: get a list of files in a given bucket. For bucket `naa-vre-user-data`, only list files in your personal folder
get_bucket_df(bucket="naa-vre-user-data", prefix=paste0(param_minio_user_prefix, "/"))


# =========================
# Helper function to download multiple files
# =========================
download_rda_files <- function(bucket, folder, files, local_path=".") {
  if (!dir.exists(local_path)) dir.create(local_path, recursive = TRUE)
  
  for (file in files) {
    object_key <- paste0(folder, "/", file)
    local_file <- file.path(local_path, file)
    cat("Downloading:", object_key, "\n")
    save_object(bucket=bucket, object=object_key, file=local_file)
    cat("✓", file, "downloaded\n")
  }
}

# =========================
# Files to download from personal TempSED folder
# =========================
# Define your MinIO endpoint and bucket name
bucket_name <- "naa-vre-user-data"  # Replace with your bucket name
minio_folder <- paste0(param_minio_user_prefix, "/TempSED/Ricky_data")  # Replace with your folder in the bucket
local_folder <- "/tmp/data/processed_results"  # Replace with the local folder path

# Ensure the local folder exists
if (!dir.exists(local_folder)) {
  dir.create(local_folder, recursive = TRUE)
}

example_data_files <- c(
  "obs2020.rda",
  "out_WD_0.002_2020.rda",
  "out_WT_0.002_2020.rda",
  "sed_pars2020.rda",
  "run_indices2020.rda",
  "Temp.ini_2020_fixedDeepT.rda"
)

# Download all required files from MinIO
print("Downloading files from MinIO...")
download_rda_files(bucket=bucket_name, folder=minio_folder, files=example_data_files)
print("All files downloaded successfully!")

# Load all data

# =========================
# Load all .rda files automatically
# =========================
print("Loading data into R...")
for (file in example_data_files) {
  load(file)
}
cat("All data loaded successfully!\n")

# =========================
# Extract columns from obs2020
# =========================
fWind.wad2020      <- obs2020[,c("Second", "windSpeed")]
fRad.wad2020       <- obs2020[,c("Second", "radiation")]
fTair.wad2020      <- obs2020[,c("Second", "airTemperature")]
fPair.wad2020      <- obs2020[,c("Second", "airPressure")]
fHumidity.wad2020  <- obs2020[,c("Second", "airHumidity")]
fCloud.wad2020     <- obs2020[,c("Second", "cloudCover")]

cat("obs2020 preview:\n")
print(head(obs2020))



,Bucket,CreationDate
,<chr>,<chr>
1,naa-vre-public,2024-01-15T15:56:11.707Z
2,naa-vre-user-data,2024-02-14T16:46:55.908Z
3,naa-vre-waddenzee-shared,2024-05-15T12:22:45.112Z


,Key,LastModified,ETag,Size,Owner_ID,Owner_DisplayName,StorageClass,Bucket
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,zhanqing2016@gmail.com/.ipynb_checkpoints/,2025-09-25T09:20:50.609Z,"""d41d8cd98f00b204e9800998ecf8427e""",0,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
2,zhanqing2016@gmail.com/.ipynb_checkpoints/Untitled-checkpoint.ipynb,2025-09-25T09:34:20.716Z,"""eac88a0c50e3652f38ee0c46f835b442""",1014,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
3,zhanqing2016@gmail.com/.ipynb_checkpoints/untitled-checkpoint.naavrewf,2025-09-25T09:37:40.608Z,"""d41d8cd98f00b204e9800998ecf8427e""",0,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
4,zhanqing2016@gmail.com/PCLake_NaaVRE_GLEON2025_Workshop/Python_notebook.ipynb,2025-01-15T16:11:38.227Z,"""95af73de1095dac91fd2f7f6848849d2""",3152,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
5,zhanqing2016@gmail.com/PCLake_NaaVRE_GLEON2025_Workshop/R_notebook.ipynb,2025-01-15T16:11:38.228Z,"""6429d34ee3b0ab7c2834c1647691152b""",11749,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
6,zhanqing2016@gmail.com/PCLake_PLoads.png,2025-02-16T09:42:25.914Z,"""8090c2a750ba1173e9c7c9796f131b7f""",136760,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
7,zhanqing2016@gmail.com/PCLake_output.csv,2025-02-16T09:42:25.401Z,"""8b815a97044e43410ba7f9b7ee622de5""",3228976,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
8,zhanqing2016@gmail.com/TempSED/Ricky_data/2100scenarios.zip,2026-01-20T10:24:15.594Z,"""16d10b356f84d172b9849637a5a6a49d-5245""",27498114879,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data
9,zhanqing2016@gmail.com/TempSED/Ricky_data/Temp.ini_2020_fixedDeepT.rda,2026-01-20T10:24:16.592Z,"""a7bd68b23efebf4d8abcd2e2def4c306""",952,02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4,minio,STANDARD,naa-vre-user-data


[1] "Downloading files from MinIO..."


[1] "obs2020.rda"

[1] "✓ obs2020.rda downloaded"


In [ ]:

path <- "/Virtual Labs/Waddenzee proto DT/Heatwave_bivalve/"
source("run_ensemble_2020.R")

# === Universal Parameters ===
z_max <- 10
dz_1     <- 1e-4
Grid = setup.grid.1D(N = 100, dx.1 = dz_1, L = z_max)

parms <- list(
  em_air = 0.8,
  em_sediment = 0.95,
  stanton = 0.001,
  dalton = 0.0014,
  density_water = 1024,
  density_solid = 2500,
  cp_water = 3994,
  tc_water = 0.6,
  tc_solid = 7,
  albedo_water = 0.05,
  kd_water = 1,
  kd_sediment = 1000
)

# === Model Run ===
output_dir <- "./output_dir/multicore/output_list_OS_2020_fixedDeepT_2yr"
#dir.create("../data/output_list_OS_2020_fixedDeepT_2yr", showWarnings = FALSE)

if(!dir.exists(output_dir)) dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)


seconds_per_year <- 365 * 24 * 3600
times_1yr <- seq(from = 3600, by = 3600, length.out = 8760)
forcing_times_1yr <- seq(from = 3600, by = 600, length.out = 52560)

delta_U_threshold <- 0.001

run_indices_sub = run_indices2020[1:3114]



cores <- as.numeric(Sys.getenv("SLURM_CPUS_PER_TASK"))
out <- mclapply(seq_along(run_indices_sub), multicore_ensemble, mc.cores = cores)
